In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import pickle
import os

Our goal is to check last black friday sale and understand
1) Our marketing department wants to know what product category purchase rate is the highest and also what product category is giving our company the highest amount of income.
2) Is there price difference between black friday sales and other days sales? We want to increese our income so our company will try to decrease sale % (A/B test required)
3) What categories of products were sold the most by state and city - we need to be sure that we have enough goods for our customers

Having this 3 objectives here is the list of steps that we need to do
1) import all related datasets
2) check all the types of imported columns
3) check all the columns for missing values
4) check for anomalies
5) rename-translate-transform data, create one big table, prepare it for analysis
6) answer the questions

Step 6 will be discussed later in details

# Import all related datasets

We need 4 tables here
1) data/olist_orders_dataset.csv - here we have order_id, order_status and order_purchase_timestamp
2) data/olist_order_items_dataset.csv - here we have order_id, product_id and price per item
3) data/olist_products_dataset.csv - here we have product_id and product_category_name

In [111]:
orders_file_path = '../data/olist_orders_dataset.csv'
order_items_file_path = '../data/olist_order_items_dataset.csv'
products_file_path = '../data/olist_products_dataset.csv'

orders_col_list = ['order_id', 'order_status', 'order_purchase_timestamp']
order_items_col_list = ['order_id', 'product_id', 'price']
products_col_list = ['product_id', 'product_category_name']

# TODO prints will be logs in .py programm
def get_filename_from_filepath(filepath):
    # TODO add another checks - windows \ path, case when file in the same dir, etc.
    position_of_last_slash = filepath.rfind('/')
    filename = filepath[position_of_last_slash + 1:]
    return filename

def import_and_check_dataframe(filepath: str, col_list):
    # TODO add different imports for different filetypes
    filename = get_filename_from_filepath(filepath)
    # TODO add try exept here
    df = pd.read_csv(filepath, usecols=col_list)

    if ~df.empty:
        print(filename + ' imported sucsessfully! Shape is', df.shape)
    else:
        print('Empty dataframe! Check file')

    return df

df_orders = import_and_check_dataframe(orders_file_path, orders_col_list)
df_order_items = import_and_check_dataframe(order_items_file_path, order_items_col_list)
df_products = import_and_check_dataframe(products_file_path, products_col_list)

all_dataframes_with_names = [['orders_dataset', df_orders], ['order_items', df_order_items], ['products',df_products]]

olist_orders_dataset.csv imported sucsessfully! Shape is (99441, 3)
olist_order_items_dataset.csv imported sucsessfully! Shape is (112650, 3)
olist_products_dataset.csv imported sucsessfully! Shape is (32951, 2)


# Check all the types of imported columns
# Plus missing values

In [112]:
def show_info_on_df(name_df_pair):
    name = name_df_pair[0]
    df = name_df_pair[1]
    print('-' * 40)
    print('general information on', name, 'df')
    print('-' * 40)
    print('non missing values and dtypes')
    print(df.info())
    print('-' * 40)
    print('only missing values')
    print(df.isnull().sum())

    print('-' * 40)


for pair in all_dataframes_with_names:
    show_info_on_df(pair)


----------------------------------------
general information on orders_dataset df
----------------------------------------
non missing values and dtypes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   order_id                  99441 non-null  object
 1   order_status              99441 non-null  object
 2   order_purchase_timestamp  99441 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB
None
----------------------------------------
only missing values
order_id                    0
order_status                0
order_purchase_timestamp    0
dtype: int64
----------------------------------------
----------------------------------------
general information on order_items df
----------------------------------------
non missing values and dtypes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 t

order_purchase_timestamp column in df_orders imported as object. We need to transform it to datetime format

We have 610 products with missing category. This is less than 2% of all products.<br>
In most cases it is reasonable to drop this data, but let's be extra safe here and be sure that this products sell rate is low (we will check it later)

In [113]:
# TODO we can create a function here. But for now it is only one column
# TODO add possible format scheme for parsing timestamp like "dd-mm-yyy HH:MM:SS"
# transform order_purchase_timestamp to datetime
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
print(df_orders.info())
print('-' * 80)

# testing that everything worked just fine
timestamp_test_obj = df_orders['order_purchase_timestamp'].iloc[0]

try:
    print(timestamp_test_obj, 
        "\nyear\t", timestamp_test_obj.year, 
        "\nmonth\t", timestamp_test_obj.month, 
        "\nday\t", timestamp_test_obj.day, 
        "\nhour\t", timestamp_test_obj.hour)
    print('everything is ok')
except Exception as e:
    print(f'something is wrong, check error: \n{e}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  99441 non-null  object        
 1   order_status              99441 non-null  object        
 2   order_purchase_timestamp  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 2.3+ MB
None
--------------------------------------------------------------------------------
2017-10-02 10:56:33 
year	 2017 
month	 10 
day	 2 
hour	 10
everything is ok


# Check anomalies

In [114]:
def calculate_statistic_info(name_df_pair):
    name = name_df_pair[0]
    df = name_df_pair[1]
    print('-' * 40)
    print('general stat information on', name, 'df')
    print('-' * 40)
    print(df.describe())
    print('-' * 40)


for pair in all_dataframes_with_names:
    calculate_statistic_info(pair)


----------------------------------------
general stat information on orders_dataset df
----------------------------------------
                                order_id order_status order_purchase_timestamp
count                              99441        99441                    99441
unique                             99441            8                    98875
top     e481f51cbdc54678b7cc49136f2d6af7    delivered      2018-04-11 10:48:14
freq                                   1        96478                        3
first                                NaN          NaN      2016-09-04 21:15:19
last                                 NaN          NaN      2018-10-17 17:30:18
----------------------------------------
----------------------------------------
general stat information on order_items df
----------------------------------------
               price
count  112650.000000
mean      120.653739
std       183.633928
min         0.850000
25%        39.900000
50%        74.990000
75%   

/tmp/ipykernel_375/670927774.py:7: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(df.describe())


Looks like no anomalies here. Some prices could be really high and 6735$ for single item is something imaginable.
Also we have data recorded for 2 years - so no 1900 year anomalies or something.
Also it seems that logging of time is pretty precise because one of the most frequent value is 2018-04-11 10:48:14 and its only appears 3 times (and it is black friday) - so no delays or something in logging order_purchase_timestamp

# Create one big table

In [115]:
# TODO here we can create function to check if our merges create duplicates - first thought: shape should remain the same

print(df_order_items.shape)
df_orders_plus = pd.merge(df_order_items, df_orders, on='order_id').merge(df_products, on='product_id')
print(df_orders_plus.shape)

# TODO remove this part, it is only for DEBUG and educational purposes
# ------------------8<-----------------------
# I'm just curious, let's check id's of this DataFrames objects. I bet df_orders_plus is in another memory cell
print('-'*40)
print(id(df_order_items))
print(id(df_orders))
print(id(df_products))
print(id(df_orders_plus))

# Nice =)
# ------------------8<-----------------------


(112650, 3)
(112650, 6)
----------------------------------------
140136314151168
140136318419056
140136313836496
140136923882736


In [116]:
# Now we can check our products with missing categories
sum_of_all_sold_products = df_orders_plus.price.sum()
mask_for_purch_wo_category = df_orders_plus['product_category_name'].isnull()

purch_wo_category_part_from_overall_purch_sum = df_orders_plus[mask_for_purch_wo_category].price.sum() / \
                                                sum_of_all_sold_products

count_of_all_sold_products = df_orders_plus.shape[0]
purch_wo_category_part_from_overall_purch_count = df_orders_plus[mask_for_purch_wo_category].shape[0] / \
                                                  count_of_all_sold_products

print('let\'s count how many products presented in db has no category',
      np.float64(df_products.product_category_name.isnull().sum()/ df_products.shape[0]).round(2),
      '\nsum purch w/o category to all prch sum percent',     
      (purch_wo_category_part_from_overall_purch_sum*100).round(2),
      '\ncount purch w/o category to all prch count percent',
      np.float64(purch_wo_category_part_from_overall_purch_count*100).round(2))

let's count how many products presented in db has no category 0.02 
sum purch w/o category to all prch sum percent 1.32 
count purch w/o category to all prch count percent 1.42


This simple analysis tels us that even there is only 0.02% of not products w/o category in our df_products dataset we can clearly see that sum and count of this products selling is more than 1%<br>
We also don't have product names in our dataset. Only categories.<br>
In real case scenario I would definitely consult with someone, who collected data or with business owner, but here I won't drop this data but I will assign "unknown category" label to it.

In [117]:
df_orders_plus['product_category_name'] = df_orders_plus['product_category_name'].fillna('unknown_category')
# I have used underscore here. every category here is using this underscore

In [118]:
df_orders_plus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 6 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   order_id                  112650 non-null  object        
 1   product_id                112650 non-null  object        
 2   price                     112650 non-null  float64       
 3   order_status              112650 non-null  object        
 4   order_purchase_timestamp  112650 non-null  datetime64[ns]
 5   product_category_name     112650 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 6.0+ MB


# Rename-translate-transform data

In [119]:
df_orders_plus.product_category_name.value_counts()
# categories are in foreign language and merged together, let's try to fix it


cama_mesa_banho                  11115
beleza_saude                      9670
esporte_lazer                     8641
moveis_decoracao                  8334
informatica_acessorios            7827
                                 ...  
cds_dvds_musicais                   14
la_cuisine                          14
pc_gamer                             9
fashion_roupa_infanto_juvenil        8
seguros_e_servicos                   2
Name: product_category_name, Length: 74, dtype: int64

In [94]:


url = "https://google-translate1.p.rapidapi.com/language/translate/v2/languages"

headers = {
	"Accept-Encoding": "application/gzip",
	"X-RapidAPI-Key": "951fd9ac96msh3b013ff9c3e72bap1e98f8jsnae389f244e8b",
	"X-RapidAPI-Host": "google-translate1.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"data":{"languages":[{"language":"af"},{"language":"ak"},{"language":"am"},{"language":"ar"},{"language":"as"},{"language":"ay"},{"language":"az"},{"language":"be"},{"language":"bg"},{"language":"bho"},{"language":"bm"},{"language":"bn"},{"language":"bs"},{"language":"ca"},{"language":"ceb"},{"language":"ckb"},{"language":"co"},{"language":"cs"},{"language":"cy"},{"language":"da"},{"language":"de"},{"language":"doi"},{"language":"dv"},{"language":"ee"},{"language":"el"},{"language":"en"},{"language":"eo"},{"language":"es"},{"language":"et"},{"language":"eu"},{"language":"fa"},{"language":"fi"},{"language":"fr"},{"language":"fy"},{"language":"ga"},{"language":"gd"},{"language":"gl"},{"language":"gn"},{"language":"gom"},{"language":"gu"},{"language":"ha"},{"language":"haw"},{"language":"he"},{"language":"hi"},{"language":"hmn"},{"language":"hr"},{"language":"ht"},{"language":"hu"},{"language":"hy"},{"language":"id"},{"language":"ig"},{"language":"ilo"},{"language":"is"},{"language":"it"

In [96]:
url = "https://google-translate1.p.rapidapi.com/language/translate/v2"

payload = "q=Hello%2C%20world!&target=es&source=en"
headers = {
	"content-type": "application/x-www-form-urlencoded",
	"Accept-Encoding": "application/gzip",
	"X-RapidAPI-Key": "951fd9ac96msh3b013ff9c3e72bap1e98f8jsnae389f244e8b",
	"X-RapidAPI-Host": "google-translate1.p.rapidapi.com"
}

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

{"data":{"translations":[{"translatedText":"¡Hola Mundo!"}]}}


ok, we have this google translate API via rapidapi.com. We can do 5 requests per second and no more 500 requests per month. So, we have like 495 requests more to finish this translation<br>
Last .value_counts() operation showed us, that we have only 74 unique categories, so we can try to create multiple dicts with translations for categories<br>
Then we need to serialize this python object to have it in file for next usages (again, we have only 500 requests per month, 5 requests per second)

In [120]:
# let's try our value
def create_payload_from_value(value, target='en', source='pt'):
    payload = "q="
    payload += value
    payload += f'&target={target}&source={source}'
    return payload

create_payload_from_value(df_orders_plus.product_category_name.unique()[2])

'q=moveis_decoracao&target=en&source=pt'

In [124]:
df_orders_plus.product_category_name.unique()[4]

'ferramentas_jardim'

In [108]:
url = "https://google-translate1.p.rapidapi.com/language/translate/v2"

payload = create_payload_from_value(df_orders_plus.product_category_name.unique()[2])
headers = {
	"content-type": "application/x-www-form-urlencoded",
	"Accept-Encoding": "application/gzip",
	"X-RapidAPI-Key": "951fd9ac96msh3b013ff9c3e72bap1e98f8jsnae389f244e8b",
	"X-RapidAPI-Host": "google-translate1.p.rapidapi.com"
}

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

{"data":{"translations":[{"translatedText":"furniture_decoration"}]}}


Looks like it worked. Let's try to create our first dict<br>
but I will save it with "if False:" just to be sure that this part is not executing automaticly

In [143]:
def translate_from_pt_to_en(str_to_translate):
    url = "https://google-translate1.p.rapidapi.com/language/translate/v2"
    headers = {
	"content-type": "application/x-www-form-urlencoded",
	"Accept-Encoding": "application/gzip",
	"X-RapidAPI-Key": "951fd9ac96msh3b013ff9c3e72bap1e98f8jsnae389f244e8b",
	"X-RapidAPI-Host": "google-translate1.p.rapidapi.com"
    }
    payload = create_payload_from_value(str_to_translate)
    
    responce = requests.request("POST", url, data=payload, headers=headers)
    parsed_translation = json.loads(responce.text)['data']['translations'][0]['translatedText']

    return parsed_translation


In [144]:
a = translate_from_pt_to_en('ferramentas_jardim')

In [152]:
print(a)
time.sleep(0.21)
print(a)

garden_tools
garden_tools


In [159]:
def translate_series_of_values(series_of_values: pd.Series):
    translation = {}
    # used [:5] here just for debug perpuses
    for element in series_of_values.unique()[:5]:
        translation[element] = translate_from_pt_to_en(element)
        # we need to sleep here because our max requests is 5 per second
        time.sleep(0.21)
    
    return translation

In [158]:
df_orders_plus.product_category_name.unique()[:5]

array(['cool_stuff', 'pet_shop', 'moveis_decoracao', 'perfumaria',
       'ferramentas_jardim'], dtype=object)

In [ ]:
def serialize(obj, file_name):

    # Overwrites any existing file
    with open(file_name, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def de_serialize(file_name):

    with open(file_name, 'rb') as input:
        obj = pickle.load(input)
        return obj

cached_path = 'data/cached/category_translate_dict.pkl'

if os.path.isfile(cached_path):
    data = de_serialize(cached_path)
else:
    data = {}
    serialize(data, cached_path)

# Prepare it for analysis

{'data': {'translations': [{'translatedText': 'garden_tools'}]}}